# Introduction to DSGE.jl

Pearl Li

December 19, 2017

## Outline

1. Notation
2. Solving, estimating, and forecasting an existing model
3. Model implementation
4. Exercise: MA(1) model
6. Conclusion

In [1]:
# Run this block immediately because DSGE and the packages it depends on will take a while 
# to compile
using DSGE

## Notation

- $y_t$ is a vector of observables at time $t$
- $s_t$ is a vector of states, including expectations of future states and lags
- $\epsilon_t$ is a vector of exogenous shocks
- $\eta_t$ is a vector of rational expectations errors
- $u_t$ is a vector of measurement error
- $\theta$ is a vector of parameters

Equilibrium conditions

$$\Gamma_0(\theta) s_t = \Gamma_1(\theta) s_{t-1} + \Psi(\theta) \epsilon_t + \Pi(\theta) \eta_t + C(\theta)$$

which are solved to give the state-space representation

$$
\begin{align*}
s_t &= T(\theta) s_{t-1} + R(\theta) \epsilon_t + C(\theta) & \epsilon_t \sim N(0, Q(\theta)) \\
y_t &= Z(\theta) s_t + D(\theta) + u_t & u_t \sim N(0, E(\theta))
\end{align*}
$$

## Using Existing Models

Let's construct an instance of the following (log-linearized) three-equation New Keynesian model:

$$
\begin{align*}
y_t - g_t &= -\frac{1}{\tau} R_t + \frac{1}{1 + \tau} \mathbb{E} \pi_{t+1} + \mathbb{E}(y_{t+1} - g_{t+1}) + \frac{1}{\tau} \mathbb{E} z_{t+1} \\
\pi_t &= \beta \mathbb{E} \pi_{t+1} + \kappa (y_t - g_t) \\
R_t &= \rho_R R_{t-1} + (1 - \rho_R) [\psi_1 \pi_t + \psi_2 (y_t - g_t)] + \epsilon_{R,t}
\end{align*}
$$

where

$$
\begin{align*}
s_t &= [y_t, \pi_t, R_t, y_{t-1}, g_t, z_t, \mathbb{E} y_{t+1}, \mathbb{E} \pi_{t+1}]' \\
y_t &= [\text{Real per-capita GDP growth}_t, \text{CPI inflation}_t, \text{Nominal FFR}_t]'
\end{align*}
$$

are the state and observable vectors respectively.

In [2]:
# Construct model object
m = AnSchorfheide()

# Set data vintage and initial forecast date
m <= Setting(:data_vintage, "170528")
m <= Setting(:date_forecast_start, DSGE.quartertodate("2017-Q2"))

# Set input and output directories
m <= Setting(:dataroot, joinpath(pwd(), "input_data"))
m <= Setting(:saveroot, pwd())

# Don't use population forecast
m <= Setting(:use_population_forecast, false)

In [ ]:
m

In [3]:
# Read in a previously computed vector of modal parameters
mode_file = rawpath(m, "estimate", "paramsmode.h5")
specify_mode!(m, mode_file)

Loaded previous mode from /Users/rcepxl10/Drive/frbny/travel/iadb-workshop/IADB_2017_Workshop/dsgejl/output_data/an_schorfheide/ss0/estimate/raw/paramsmode_vint=170528.h5.


In [ ]:
# Solve the rational expectations model to get transition equation 
# matrices
TTT, RRR, CCC = solve(m)

Estimate and forecast the model (don't run this code block!):

In [ ]:
# Find a posterior mode, sample from the posterior distribution
estimate(m)

# Add parallel workers
my_procs = addprocs(50)

# Forecast using the full distribution of parameters
output_vars = [:forecaststates, :forecastobs]
forecast_one(m, :full, :none, output_vars)
compute_meansbands(m, :full, :none, output_vars)

# Remove parallel workers
rmprocs(my_procs)

Since in practice estimating and forecasting the full distribution is time-consuming, we'll forecast only at the mode using the mode we read in:

In [4]:
# Load data
df = load_data(m)

# Forecast using modal parameters
output_vars = [:histobs, :forecaststates, :forecastobs]
forecast_one(m, :mode, :none, output_vars, df = df, verbose = :none)
compute_meansbands(m, :mode, :none, output_vars, verbose = :none)

Reading dataset /Users/rcepxl10/Drive/frbny/travel/iadb-workshop/IADB_2017_Workshop/dsgejl/input_data/data/data_dsid=00_vint=170528.csv from disk...dataset from disk valid


In [ ]:
# Read in forecasted observables
mb = read_mb(m, :mode, :none, :forecastobs)

In [ ]:
# Show first four forecasted quarters
mb.means[1:4, :]

In [19]:
# Read in forecasted states
mb = read_mb(m, :mode, :none, :forecaststates)

# Show first forecasted quarters of output, inflation, and interest rate
mb.means[1:4, [:y_t, :π_t, :R_t]]

,y_t,π_t,R_t
1,0.8118952136027373,0.2350315239936614,-0.2356874614957492
2,0.7161825375221876,0.16661955296739311,-0.14711159658096123
3,0.6486366042580868,0.12011639949265113,-0.08787651429325914
4,0.600145688870977,0.08840842601171206,-0.04841225410162462


In [15]:
# Plot GDP growth history and modal forecast
plot_history_and_forecast(m, :obs_gdp, :obs, :mode, :none, 
    start_date = DSGE.quartertodate("2007-Q1"),
    end_date   = DSGE.quartertodate("2020-Q4"))

Saved /Users/rcepxl10/Drive/frbny/travel/iadb-workshop/IADB_2017_Workshop/dsgejl/output_data/an_schorfheide/ss0/forecast/figures/forecast_obs_gdp_cond=none_para=mode_vint=170528.html


## The Model Object

Subtypes of `AbstractModel` contain the following fields:

In [16]:
fieldnames(m)

18-element Array{Symbol,1}:
 :parameters                 
 :steady_state               
 :keys                       
 :endogenous_states          
 :exogenous_shocks           
 :expected_shocks            
 :equilibrium_conditions     
 :endogenous_states_augmented
 :observables                
 :pseudo_observables         
 :spec                       
 :subspec                    
 :settings                   
 :test_settings              
 :rng                        
 :testing                    
 :observable_mappings        
 :pseudo_observable_mappings 

These fields include:

- Vectors of time-invariant (`parameters`) and steady-state parameters (`steady_state`)
- `Dict{Symbol, Int}`s mapping states, shocks, expectational errors, equations, or observables to indices. For example:

In [17]:
m.endogenous_states

DataStructures.OrderedDict{Symbol,Int64} with 8 entries:
  :y_t   => 1
  :π_t   => 2
  :R_t   => 3
  :y_t1  => 4
  :g_t   => 5
  :z_t   => 6
  :Ey_t1 => 7
  :Eπ_t1 => 8

`m.endogenous_states[:y_t] = 1` indicates that the first element of the state vector

$$s_t = [y_t, \pi_t, R_t, y_{t-1}, g_t, z_t, \mathbb{E} y_{t+1}, \mathbb{E} \pi_{t+1}]$$

is $y_t$.

In [18]:
m.equilibrium_conditions

DataStructures.OrderedDict{Symbol,Int64} with 8 entries:
  :eq_euler    => 1
  :eq_phillips => 2
  :eq_mp       => 3
  :eq_y_t1     => 4
  :eq_g        => 5
  :eq_z        => 6
  :eq_Ey       => 7
  :eq_Eπ       => 8

`m.equilibrium_conditions[:eq_euler] = 1` means the first row of the equilibrium conditions

$$\Gamma_0 s_t = \Gamma_1 s_{t-1} + \Psi \epsilon_t + \Pi \eta_t + C$$

is the consumption Euler equation.

Additional fields in the model object:

- Strings giving the model specification (`spec`) and subspecification (`subspec`)
- `Dict{Symbol, Setting}`s of model settings, both for regular use (`settings`) and testing the package (`test_settings`)

## Exercise: MA(1) Model

See `exercise.ipynb`

## Conclusion

Things learned:

- We love open source (and Julia)!
  + Open-source languages and packages reduce costs of writing code and make it easier to share
  + Julia is high-performance and high-productivity
- Challenges to be aware of
  + New language, frequent updates: This will slow down when v1.0 comes out (hopefully this year)
  + Sparse StackOverflow activity

Ongoing work:

- Forecasting under alternative monetary policy rules
- Forecast evaluation and decomposing changes in forecasts
- Estimating nonlinear models using the tempered particle filter ([Herbst & Schorfheide 2017](http://www.nber.org/papers/w23448))

### Thank you!

https://github.com/FRBNY-DSGE/DSGE.jl